## Face Recognition

Face recognition is the task of recognising faces. However, recognising faces can mean several things. On the one hand, it can mean that faces can be assigned to known persons. These known persons are then stored in a database and for each picture of a face it can then be said whether it is, for example, Olaf Scholz or Christian Lindner. Another method of recognition is that you don't even know which person a face belongs to, but you can say that two faces belong to the same person.

![Face Recognition with Database](images/FaceRecognitionDatabase.png)

As we don't want to create a large database that might have to store sensitive data, we decided that our WebApp should only be able to tell whether two pictures show the same person.
To do this, we used the [face-recognition library](https://github.com/ageitgey/face_recognition) . This library offers many useful methods for face recognition. The library uses the face recognition model from [dlib](http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2) to compare the faces. The model calculates a 128D vector from the keypoints of the face, which represents the characteristics of the face. Two of these vectors can then be checked for similarity using the Euclidean distance. If the distance is below a defined threshold value, it is assumed that the person is the same. The model has achieved an accuracy of 99.38% on the standard LFW face recognition benchmark. Dlib also provides a small [demo](http://dlib.net/face_recognition.py.html) if you want to test the model yourself.

The face-recognition library provides two different methods for face recognition. The first method is based on five keypoints, taking only two keypoints from both eyes and one keypoint from the nose. The other method is based on 68 keypoints and uses the same keypoints that we use for the filters. As the difference in speed is very small, but the accuracy of 68 keypoints was better, we decided in favour of the 68 keypoints approach.

The whole process is shown in this diagram:

![Process of face recognition](images/FaceRecognitionProcess.png)


## Practical Application

Our python implementation for the face recognition is using the following Code in the backend:

In [1]:
def recognize_faces(orig_img: Image, mod_img: Image, orig_keypoints):
    orig_img_bgr = cv2.cvtColor(np.array(orig_img), cv2.COLOR_RGB2BGR)
    mod_img_bgr = cv2.cvtColor(np.array(mod_img), cv2.COLOR_RGB2BGR)

    gray_image = cv2.cvtColor(np.asarray(mod_img_bgr), cv2.COLOR_BGR2GRAY)
    faces = hog_svm_detector(gray_image)
    face_encodings_unknown = []
    boxes_mod = []
    for face in faces:
        box = [face.left(), face.top(), face.width(), face.height()]
        face_encodings_unknown.append(np.array(calculate_face_encoding(np.asarray(mod_img_bgr), box)))
        boxes_mod.append((face.left(), face.top(), face.width(), face.height()))

    face_encodings_orig = []
    boxes_orig = []
    for box, _, _, face_encoding_orig in orig_keypoints:
        face_encodings_orig.append(np.array(face_encoding_orig))
        boxes_orig.append(box)

    count_of_matches = 0

    for j, face_encoding_unknown in enumerate(face_encodings_unknown):
        matches = face_recognition.compare_faces(face_encodings_orig, face_encoding_unknown, tolerance=0.55)

        for i, match in enumerate(matches):
            if match:
                count_of_matches += 1
                selected_color = palette[(j * 2) % num_colors]
                bgr_color = tuple(int(value * 255) for value in selected_color)
                box_orig = boxes_orig[i]
                box_mod = boxes_mod[j]
                top, right, bottom, left = box_orig[1], box_orig[0] + box_orig[2], box_orig[1] + box_orig[3], box_orig[
                    0]
                cv2.rectangle(orig_img_bgr, (left, top), (right, bottom), bgr_color, 6)

                top, right, bottom, left = box_mod[1], box_mod[0] + box_mod[2], box_mod[1] + box_mod[3], box_mod[0]
                cv2.rectangle(mod_img_bgr, (left, top), (right, bottom), bgr_color, 6)

    orig_img_rgb = cv2.cvtColor(orig_img_bgr, cv2.COLOR_BGR2RGB)
    mod_img_rgb = cv2.cvtColor(mod_img_bgr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(orig_img_rgb), Image.fromarray(mod_img_rgb), count_of_matches Code

SyntaxError: invalid syntax (341758604.py, line 41)

This implementation takes an original image with the calculated keypoints, which also contains the 128D vector of the characteristics of the face and a potential modified image that is compared to the original image. The modified image is pre-processed and then the face bounding boxes are calculated using the svm+hog face detection algorithm. The bounding boxes are then used to compute the 128D vector of the faces found in the image. 

Each face found in the modified image is then compared to each face in the original image. If the Euclidean distance is less than the given tolerance of 0.55, a match is registered. For each match, a pair of colored boxes is drawn in both of the images where the matching faces are. The number of pairs is also counted.

Here is an example output of the algorithm:

![Example of a recognized faces](images/detected-faces-examples/example_face_recognition.png)